In [19]:
import linecache
import numpy as np
import matplotlib.pyplot as plt
import charts
import os
from IPython.display import display, clear_output
from IPython.html    import widgets
import random
import json

def line_index(fileName,words):
    global dose_idx, last_line_idx
    last_line_idx = 0    
    with open(fileName) as f:
        for line_number, line in enumerate(f,1):
            word = line.strip()
            for word in words:
                if line.find(word)>=0:
                    data1, name = line.split (': ')
                    name = name[:-1]
                    structures.append((line_number,name.upper()))


def dvh_v_rel(str_idx,dose):
    return round(np.interp(dose,total_dvh[str_idx][2],total_dvh[str_idx][3]),2)

def dvh_d_rel(str_idx,vol):
    return round(np.interp(vol,total_dvh[str_idx][3][::-1],total_dvh[str_idx][2][::-1]),2)

def dvh_d_abs(str_idx,vol):
    rel_vol = vol*100.0 / total_dvh[3][1]
    ret = round(dvh_d_rel(str_idx,rel_vol),2)
    return ret

def gEUD(dose_cum,vol_cum, a = 1):
    if a == 0:
        return "Error, 'a' must be non-zero"
    elif len(dose_cum) != len(vol_cum):
        return "Error different length of dose and volume arrays"
    elif vol_cum[-1]>0:
        return "Entire volume not covered in dose range"
    elif a>100 or a<-100:
        return "Error 'a' is out of range"
    diff_dose = []
    res = 0
    diff_vol = -1 * np.diff(vol_cum)
    for idx in range(len(dose_cum)-1):
        diff_dose.append((dose_cum[idx]+dose_cum[idx+1])/2.0)
    for idx in range(len(diff_dose)):
        res +=diff_dose[idx]**a * diff_vol[idx]/100.0
    
    res = res**(1.0/a)
    return res

In [20]:
filenames=[]
#dir_name=os.getcwd()
#dir_pinn = str(dir_name + './DVHpinn')
dir_name = './data'
for root, dirs, files in os.walk(dir_name):
    for file in files:
        filenames.append(file)


pts_list = widgets.Select(options = [x for x in filenames],descrition = "Patient List")
display(pts_list)

In [22]:
structures = []
words2 = ["Roi:"]
fileName2 = os.path.join(dir_name, pts_list.value)                   
                   
line_index(fileName2,words2)

In [27]:
total_dvh=[]

list_size = structures[1][0]-structures[0][0]-1
for idx in structures:
    bin_size = float(linecache.getline(fileName2,structures[0][0]-2).strip(' \t\n\r').split()[2])    
    start = idx[0]+2
    stop = idx[0]+list_size-1
    vol_in_cc = float(linecache.getline(fileName2,idx[0]+2).split("\t")[1]) 
    dvh_dose = []
    dvh_vol = []
    while start < stop:
        [dose_bin, vol] = linecache.getline(fileName2,start).strip(' \t\n\r').split()      
        dose = float((float(dose_bin)-1)*bin_size/100.0)
        rel_vol = float(vol)/vol_in_cc*100.0
        dvh_dose.append(dose)
        dvh_vol.append(rel_vol)
        start +=1
    
    total_dvh.append((idx[1],vol_in_cc,dvh_dose,dvh_vol))

dvh_series = []

for idx in range(len(total_dvh)):
    s = dict(name=total_dvh[idx][0], data = zip(total_dvh[idx][2],total_dvh[idx][3]))
    dvh_series.append(s)
    
options = {
    'title':{'text':'Cumulative DVH'},
    'yAxis':{
        'title':{'text':'Volume (%)'},
        'min':0,
        'max':100
    },
    
    'xAxis':{
        'title':{'text':'Dose (Gy)'}
    },
    'chart':{'zoomType':'xy'},
    'plotOptions':{'spline':{'marker':{'enabled':False}}
             }
}
    
charts.plot(dvh_series, options=options, show='inline', type='spline')

In [30]:
volumes = [x[0] for x in total_dvh]
volumes.insert(0,"Not Available")
protocols = ["HDR-4x7Gy","PDR-50pulses","PDR-30pulses"]


protocol= widgets.Dropdown(options=protocols,description = "Dose (Gy):")
hr_ctv = widgets.Dropdown(options=volumes, description = "HR-CTV: ")
ir_ctv = widgets.Dropdown(options=volumes, description = "IR-CTV: ")
gtv = widgets.Dropdown(options=volumes, description = "GTV: ")
rectum = widgets.Dropdown(options=volumes, description = "Rectum: ")
bladder = widgets.Dropdown(options=volumes, description = "Bladder: ")
sigmoid = widgets.Dropdown(options=volumes, description = "Sigmoid: ")
small_bowels = widgets.Dropdown(options=volumes, description = "Small Bowels: ")





display(protocol,hr_ctv,ir_ctv,gtv,rectum,bladder,sigmoid,small_bowels)